# Data Load and Preprocessing for Accident 'Crash-Stats'

The purpose of this notebook is to access the Victoria Police 'Crash Stats' datasets hosted on the VicRoads open data platform, and join these datasets together to form a view of pedestrian accidents in Melbourne metropolitan and CBD between 01/01/2008 and end of file which is 31/03/2020.

In [1]:
#Use google drive as the data repository
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Read in all log files within the zip file.
from ipywidgets import IntProgress
import pandas as pd
import numpy as np
from IPython.display import display
from zipfile import ZipFile
#fnmatch helps to filter for specific file types
import fnmatch
import os
from pathlib import Path
import datetime
datetime.datetime.strptime

<function datetime.strptime>

# Load and Unpack the Accident 'Crash-Stats' Zip File

In [3]:
!wget "https://vicroadsopendatastorehouse.vicroads.vic.gov.au/opendata/Road_Safety/ACCIDENT.zip" -P "/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data"

--2022-08-06 08:56:37--  https://vicroadsopendatastorehouse.vicroads.vic.gov.au/opendata/Road_Safety/ACCIDENT.zip
Resolving vicroadsopendatastorehouse.vicroads.vic.gov.au (vicroadsopendatastorehouse.vicroads.vic.gov.au)... 104.17.50.85, 104.17.49.85, 2606:4700::6811:3255, ...
Connecting to vicroadsopendatastorehouse.vicroads.vic.gov.au (vicroadsopendatastorehouse.vicroads.vic.gov.au)|104.17.50.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36176162 (34M) [application/zip]
Saving to: ‘/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data/ACCIDENT.zip’

ACCIDENT.zip        100%[===================>]  34.50M  16.7MB/s    in 2.1s    

2022-08-06 08:56:39 (16.7 MB/s) - ‘/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data/ACCIDENT.zip’ saved [36176162/36176162]



In [4]:
# working directory
path = Path('/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data')

# source directory containing files to zip
src_dir   = path

# csv filepath (to be created/overwritten)
csv_dst   = path / 'accident_data.csv'

# zip filepath (to be created/overwritten)
zip_dst   = path / 'ACCIDENT.zip'

# unzip directory (to be created/overwritten)
unzip_dst = path / 'raw'

In [5]:
unzip_dst.mkdir(parents=True, exist_ok=True)
unzip_dst.exists(), unzip_dst

(True,
 PosixPath('/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data/raw'))

In [6]:
with ZipFile(zip_dst, 'r') as zf:
    zf.printdir() # print zip contents
    zf.extractall(unzip_dst)

File Name                                             Modified             Size
ACCIDENT.csv                                   2020-11-04 14:25:42     55566147
ACCIDENT_CHAINAGE.csv                          2020-11-04 14:25:42      7095110
ACCIDENT_EVENT.csv                             2020-11-04 14:25:42     36690855
ACCIDENT_LOCATION.csv                          2020-11-04 14:25:42     17626667
ATMOSPHERIC_COND.csv                           2020-11-04 14:25:42      6743142
NODE.csv                                       2020-11-04 11:04:04     30205255
NODE_ID_COMPLEX_INT_ID.csv                     2020-11-04 14:25:42      5705980
PERSON.csv                                     2020-11-04 14:25:42     53201638
ROAD_SURFACE_COND.csv                          2020-11-04 14:25:42      6195425
Statistic Checks.csv                           2020-11-04 14:25:42         3949
SUBDCA.csv                                     2020-11-04 14:25:42     14687904
VEHICLE.csv                             

# Data Preparation and Preprocessing

From Person.csv we need to filter 'Road User Type Desc' = 'pedestrian' to identify the correct accident number and person identifier keys

From Accident.csv we need to filter accidentdate >= 01-01-2008

From NODE.csv we need to identify geographical region LGA_NAME of 'MELBOURNE'

MERGE these three subsets together using inner joins in a sequence to construct the 'source of truth' dataset for Bicyclist accidents occuring in the city of Melbourne between 2008 and 2020.

The next step after this will be to collect/assemble additional features and variables which further describe the accident event. Datasets will be left joined on using accident number.

In [7]:
# DOMAIN: PERSON
raw_person = pd.read_csv('/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data/raw/PERSON.csv')
raw_person.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6,9,10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ACCIDENT_NO,PERSON_ID,VEHICLE_ID,SEX,AGE,Age Group,INJ_LEVEL,Inj Level Desc,SEATING_POSITION,HELMET_BELT_WORN,ROAD_USER_TYPE,Road User Type Desc,LICENCE_STATE,PEDEST_MOVEMENT,POSTCODE,TAKEN_HOSPITAL,EJECTED_CODE
0,T20060000010,01,A,F,NaN,unknown,4,Not injured,LF,1,3,Passengers,,0,3130.0,,0
1,T20060000010,02,C,M,43.0,40-49,4,Not injured,LF,1,3,Passengers,,0,7310.0,,0
2,T20060000010,03,C,M,22.0,22-25,4,Not injured,LR,1,3,Passengers,,0,7310.0,,0
3,T20060000010,A,A,M,72.0,70+,4,Not injured,D,1,2,Drivers,V,0,3130.0,,0
4,T20060000010,B,B,F,62.0,60-64,3,Other injury,D,1,2,Drivers,V,0,NaN,N,0


In [8]:
#Check starting point row counts
print('Before Filtering:', len(raw_person) )
#Select columns
wrk_person = raw_person[['ACCIDENT_NO', 'PERSON_ID', 'Road User Type Desc', 'TAKEN_HOSPITAL', 'Inj Level Desc', 'Age Group', 'POSTCODE', 'SEX']]
#Filter
wrk_person = wrk_person[wrk_person['Road User Type Desc'].str.contains('Pedestrian')]
# get distinct values of the dataframe based on column Accident Number, keep the first duplicate
wrk_person = wrk_person.drop_duplicates(keep='first', subset = ["PERSON_ID", 'ACCIDENT_NO']) #This will duplicate based on accident number but not on person id

#rename the columns (inplace)
wrk_person = wrk_person.rename(columns={'Road User Type Desc':'CATRoadUserTypeDesc_person',
                                        'TAKEN_HOSPITAL':'CATTakenHospital_person',
                                        'Inj Level Desc':'CATInjuryLevelDesc_person',
                                        'Age Group':'CATAgeGroup_person',
                                        'POSTCODE':'CATPostcode_person',
                                        'PERSON_ID':'KEYPersonID_person',
                                        'SEX': 'CATGender_person'})

#Check post-operation row counts
print('After Filtering:', len(wrk_person) )

Before Filtering: 490948
After Filtering: 21284


# Data Structure and Quality Observations:
There will be some accidents with multiple bicyclists involved. This means, one accident ID will related to two or more person IDs.

Should we want to observe events that occur, use distinct accident ID.
Should we want to observe people impacts of accidents, use distinct person ID

In [9]:
wrk_person.head()

,ACCIDENT_NO,KEYPersonID_person,CATRoadUserTypeDesc_person,CATTakenHospital_person,CATInjuryLevelDesc_person,CATAgeGroup_person,CATPostcode_person,CATGender_person
26,T20060000046,01,Pedestrians,N,Other injury,22-25,3137.0,F
34,T20060000066,01,Pedestrians,Y,Serious injury,22-25,3806.0,M
102,T20060000242,01,Pedestrians,Y,Serious injury,30-39,3175.0,M
254,T20060000506,01,Pedestrians,Y,Serious injury,40-49,3029.0,F
260,T20060000524,01,Pedestrians,N,Other injury,0-4,3031.0,M


In [10]:
# DOMAIN: ACCIDENT
raw_accident = pd.read_csv('/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data/raw/ACCIDENT.csv')
raw_accident.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ACCIDENT_NO,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,...,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,NO_PERSONS_NOT_INJ,POLICE_ATTEND,ROAD_GEOMETRY,Road Geometry Desc,SEVERITY,SPEED_ZONE
0,T20060000010,13/01/2006,12:42:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,6,0,1,0,5,1,1,Cross intersection,3,60
1,T20060000018,13/01/2006,19:10:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,4,0,1,0,3,1,2,T intersection,3,70
2,T20060000022,14/01/2006,12:10:00,7,Fall from or in moving vehicle,7,Saturday,190,FELL IN/FROM VEHICLE,MEL,...,2,1,0,0,1,1,5,Not at intersection,2,100
3,T20060000023,14/01/2006,11:49:00,1,Collision with vehicle,7,Saturday,130,REAR END(VEHICLES IN SAME LANE),MEL,...,2,1,0,0,1,1,2,T intersection,2,80
4,T20060000026,14/01/2006,10:45:00,1,Collision with vehicle,7,Saturday,121,RIGHT THROUGH,MEL,...,3,0,3,0,0,1,5,Not at intersection,3,50


In [11]:
#Check starting point row counts
print( 'Before Filtering:', len(raw_accident) )
#Select columns
wrk_accident = raw_accident[['ACCIDENT_NO', 'ACCIDENTDATE', 'ACCIDENTTIME', 'Accident Type Desc', 'Day Week Description',
                             'DCA Description', 'PAGE', 'GRID_REFERENCE_X', 'GRID_REFERENCE_Y','Light Condition Desc', 
                             'NO_OF_VEHICLES', 'NO_PERSONS', 'NO_PERSONS_INJ_2']]

#Set data types
wrk_accident['PAGE'] = wrk_accident['PAGE'].astype('|S') # by default set the length to the max len it encounters
wrk_accident['PAGE'] = wrk_accident['PAGE'].str.decode("utf-8")

#Filter
wrk_accident['ACCIDENTDATE'] = pd.to_datetime(wrk_accident['ACCIDENTDATE'], format='%d/%m/%Y')

#Filter Accident dates
wrk_accident = wrk_accident.loc[wrk_accident['ACCIDENTDATE'].dt.date >= datetime.date(2008,1,1)]

#rename the columns (inplace)
wrk_accident = wrk_accident.rename(columns={'ACCIDENTDATE':'DATAccidentDate_accident',
                                        'ACCIDENTTIME':'TIMAccidentTime_accident',
                                        'Accident Type Desc': 'CATAccidentTypeDesc_accident',
                                        'Day Week Description': 'CATDayOfWeek_accident', 
                                        'DCA Description': 'CATDCADesc_accident',
                                        'PAGE': 'CATMelwaysPage_accident',
                                        'GRID_REFERENCE_X': 'CATMelwaysGridRef_X_accident',
                                        'GRID_REFERENCE_Y': 'CATMelwaysGridRef_Y_accident',
                                        'Light Condition Desc': 'CATLightConditionDesc_accident',
                                        'NO_OF_VEHICLES':'NUMVehiclesInvolved_accident',
                                        'NO_PERSONS': 'NUMPersonsInvolved_accident',
                                        'NO_PERSONS_INJ_2': 'NUMPersonsInjured_accident'
                                        })

# get distinct values of the dataframe based on column Accident Number, keep the first duplicate
wrk_accident = wrk_accident.drop_duplicates(keep='first', subset = ["ACCIDENT_NO"])

#Check post-operation row counts
print( 'After Filtering:', len(wrk_accident) )

Before Filtering: 203708


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

After Filtering: 176157


In [12]:
wrk_accident.head()

,ACCIDENT_NO,DATAccidentDate_accident,TIMAccidentTime_accident,CATAccidentTypeDesc_accident,CATDayOfWeek_accident,CATDCADesc_accident,CATMelwaysPage_accident,CATMelwaysGridRef_X_accident,CATMelwaysGridRef_Y_accident,CATLightConditionDesc_accident,NUMVehiclesInvolved_accident,NUMPersonsInvolved_accident,NUMPersonsInjured_accident
27236,T20080000002,2008-01-01,00:12:00,Collision with vehicle,Tuesday,U TURN,65,B,6,Dark Street lights on,2,2,1
27242,T20080000013,2008-01-01,01:18:00,Struck Pedestrian,Tuesday,PED EMERGES FROM IN FRONT OF PARKED OR STATION...,2J,K,11,Dark Street lights on,1,2,1
27243,T20080000014,2008-01-01,02:10:00,Collision with vehicle,Tuesday,CROSS TRAFFIC(INTERSECTIONS ONLY),1A,F,6,Dark Street lights on,2,4,1
27244,T20080000016,2008-01-01,00:15:00,Collision with vehicle,Tuesday,HEAD ON (NOT OVERTAKING),62,D,7,Dark No street lights,2,5,4
27245,T20080000019,2008-01-01,00:50:00,Collision with vehicle,Tuesday,REAR END(VEHICLES IN SAME LANE),9,E,6,Dark Street lights on,2,3,0


In [13]:
# DOMAIN: NODE
raw_node= pd.read_csv('/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data/raw/NODE.csv')
raw_node.head()

,ACCIDENT_NO,NODE_ID,NODE_TYPE,VICGRID94_X,VICGRID94_Y,LGA_NAME,LGA_NAME_ALL,REGION_NAME,DEG_URBAN_NAME,Lat,Long,POSTCODE_NO
0,T20060002689,22,I,2495701.925,2411599.135,MELBOURNE,MELBOURNE,METROPOLITAN NORTH WEST REGION,MELB_URBAN,-37.796596,144.951197,3051
1,T20060010827,22,I,2495701.925,2411599.135,MELBOURNE,MELBOURNE,METROPOLITAN NORTH WEST REGION,MELB_URBAN,-37.796596,144.951197,3051
2,T20060017279,22,I,2495701.925,2411599.135,MELBOURNE,MELBOURNE,METROPOLITAN NORTH WEST REGION,MELB_URBAN,-37.796596,144.951197,3051
3,T20060041762,22,I,2495701.925,2411599.135,MELBOURNE,MELBOURNE,METROPOLITAN NORTH WEST REGION,MELB_URBAN,-37.796596,144.951197,3051
4,T20060047478,22,I,2495701.925,2411599.135,MELBOURNE,MELBOURNE,METROPOLITAN NORTH WEST REGION,MELB_URBAN,-37.796596,144.951197,3051


In [14]:
#Check starting point row counts
print( 'Before Filtering:', len(raw_node) )
#Select columns
wrk_node = raw_node[['ACCIDENT_NO', 'LGA_NAME', 'DEG_URBAN_NAME', 'Lat', 'Long', 'POSTCODE_NO']]
#Filter LGA NAME and DEG_URBAN_NAME using these together as missing data is present
wrk_node['LGA_NAME'] = wrk_node['LGA_NAME'].astype('|S') # by default set the length to the max len it encounters
wrk_node['LGA_NAME'] = wrk_node['LGA_NAME'].str.decode("utf-8")
wrk_node['DEG_URBAN_NAME'] = wrk_node['DEG_URBAN_NAME'].astype('|S') # by default set the length to the max len it encounters
wrk_node['DEG_URBAN_NAME'] = wrk_node['DEG_URBAN_NAME'].str.decode("utf-8")

#Conditional filter
wrk_node = wrk_node[(wrk_node['LGA_NAME'].str.contains('MELBOURNE'))]

#rename the columns (inplace)
wrk_node = wrk_node.rename(columns={'LGA_NAME':'CATLGAName_node',
                                        'DEG_URBAN_NAME':'CATDEGUrbanName_node',
                                        'Lat': 'NUMLatitude_node',
                                        'Long': 'NUMLongitude_node', 
                                        'POSTCODE_NO': 'CATPostcode_node'
                                        })

# get distinct values of the dataframe based on column Accident Number, keep the first duplicate
wrk_node = wrk_node.drop_duplicates(keep='first', subset = ["ACCIDENT_NO"])

#Check post-operation row counts
print( 'After Filtering:', len(wrk_node) )

Before Filtering: 221797


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

After Filtering: 11658


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [15]:
wrk_node.head()

,ACCIDENT_NO,CATLGAName_node,CATDEGUrbanName_node,NUMLatitude_node,NUMLongitude_node,CATPostcode_node
0,T20060002689,MELBOURNE,MELB_URBAN,-37.796596,144.951197,3051
1,T20060010827,MELBOURNE,MELB_URBAN,-37.796596,144.951197,3051
2,T20060017279,MELBOURNE,MELB_URBAN,-37.796596,144.951197,3051
3,T20060041762,MELBOURNE,MELB_URBAN,-37.796596,144.951197,3051
4,T20060047478,MELBOURNE,MELB_URBAN,-37.796596,144.951197,3051


# Create the 'Source of Truth' Dataset
Merge (inner join) Person and Accident where accident number matches in both tables. Then merge (inner join) Node where accident number matches in both tables. 

In [18]:
#First inner join for pedestrians and accident dates
wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_1 = pd.merge(wrk_accident, wrk_person, how="inner", on=["ACCIDENT_NO"])

#Second inner join for first join result and filter again for accidents occuring within Melbourne CBD specifically (data quality double-check)
wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_2 = pd.merge(wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_1, wrk_node, how="inner", on=["ACCIDENT_NO"])
wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_2.head()

,ACCIDENT_NO,DATAccidentDate_accident,TIMAccidentTime_accident,CATAccidentTypeDesc_accident,CATDayOfWeek_accident,CATDCADesc_accident,CATMelwaysPage_accident,CATMelwaysGridRef_X_accident,CATMelwaysGridRef_Y_accident,CATLightConditionDesc_accident,...,CATTakenHospital_person,CATInjuryLevelDesc_person,CATAgeGroup_person,CATPostcode_person,CATGender_person,CATLGAName_node,CATDEGUrbanName_node,NUMLatitude_node,NUMLongitude_node,CATPostcode_node
0,T20080000929,2008-01-09,08:10:00,Struck Pedestrian,Wednesday,FAR SIDE. PED HIT BY VEHICLE FROM THE LEFT ...,1B,P,10,Day,...,N,Other injury,26-29,3004.0,F,MELBOURNE,MELB_URBAN,-37.817147,144.968489,3000
1,T20080001163,2008-01-11,15:35:00,Collision with vehicle,Friday,REAR END(VEHICLES IN SAME LANE),2E,B,1,Day,...,Y,Serious injury,40-49,3064.0,M,MELBOURNE,MELB_URBAN,-37.809325,144.934021,3003
2,T20080001766,2008-01-16,14:24:00,Struck Pedestrian,Wednesday,PED NEAR SIDE. PED HIT BY VEHICLE FROM THE RIG...,1B,V,3,Day,...,N,Other injury,30-39,NaN,F,MELBOURNE,MELB_URBAN,-37.809404,144.972140,3000
3,T20080002335,2008-01-19,20:00:00,Struck Pedestrian,Saturday,ANY MANOEUVRE INVOLVING PED NOT INCLUDED IN DC...,2F,H,8,Dusk/Dawn,...,N,Other injury,26-29,3193.0,F,MELBOURNE,MELB_URBAN,-37.823070,144.971040,3004
4,T20080002388,2008-01-21,14:45:00,Struck Pedestrian,Monday,PED NEAR SIDE. PED HIT BY VEHICLE FROM THE RIG...,2A,F,9,Day,...,Y,Serious injury,64-69,3636.0,M,MELBOURNE,MELB_URBAN,-37.802386,144.943606,3051


In [19]:
wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_2.shape

(2028, 25)

# Join additional features and attributes
With the source of truth dataset constructed, left join on other raw accident datasets and obtain further useful variables/features which can further describe the accident event and circumstances.

In [20]:
#Prepare the raw datasets and set column selections

#SURFACE CONDITIONS
raw_road_surface_condition = pd.read_csv('/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data/raw/ROAD_SURFACE_COND.csv')
wrk_road_surface_condition = raw_road_surface_condition[['ACCIDENT_NO', 'Surface Cond Desc']]
wrk_road_surface_condition = wrk_road_surface_condition.drop_duplicates(keep='first', subset = ["ACCIDENT_NO"])
#rename the columns (inplace)
wrk_road_surface_condition = wrk_road_surface_condition.rename(columns={'Surface Cond Desc': 'CATSurfaceConditionDesc_surface'})

#SUB DCA
raw_subDCA = pd.read_csv('/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data/raw/SUBDCA.csv')
wrk_subDCA = raw_subDCA[['ACCIDENT_NO', 'Sub Dca Code Desc']]
wrk_subDCA = wrk_subDCA.drop_duplicates(keep='first', subset = ["ACCIDENT_NO"])
#rename the columns (inplace)
wrk_subDCA = wrk_subDCA.rename(columns={'Sub Dca Code Desc': 'CATSubDCACodeDesc_subdca'})

#ATMOSPHERIC CONDITIONS
raw_amosphericcond = pd.read_csv('/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data/raw/ATMOSPHERIC_COND.csv')
wrk_amosphericcond = raw_amosphericcond[['ACCIDENT_NO', 'Atmosph Cond Desc']]
wrk_amosphericcond = wrk_amosphericcond.drop_duplicates(keep='first', subset = ["ACCIDENT_NO"])
#rename the columns (inplace)
wrk_amosphericcond = wrk_amosphericcond.rename(columns={'Atmosph Cond Desc': 'CATAtmosphericConditionDesc_atmosphere'})

#ACCIDENT LOCATION
raw_accidentlocation = pd.read_csv('/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data/raw/ACCIDENT_LOCATION.csv')
wrk_accidentlocation = raw_accidentlocation[['ACCIDENT_NO', 'ROAD_NAME','ROAD_NAME_INT','ROAD_TYPE','ROAD_TYPE_INT']]
wrk_accidentlocation = wrk_accidentlocation.drop_duplicates(keep='first', subset = ["ACCIDENT_NO"])
#rename the columns (inplace)
wrk_accidentlocation = wrk_accidentlocation.rename(columns={'ROAD_NAME': 'CATRoadName_acclocation',
                                                            'ROAD_NAME_INT': 'CATRoadNameInt_acclocation',
                                                            'ROAD_TYPE': 'CATRoadType_acclocation',
                                                            'ROAD_TYPE_INT': 'CATRoadTypeInt_acclocation'
                                                            })
#ACCIDENT EVENT
raw_accidentevent = pd.read_csv('/content/drive/MyDrive/Uni Assignments/SIT782 Team Project B/data/raw/ACCIDENT_EVENT.csv')
wrk_accidentevent = raw_accidentevent[['ACCIDENT_NO', 'Event Type Desc','Object Type Desc']]
wrk_accidentevent = wrk_accidentevent.drop_duplicates(keep='first', subset = ["ACCIDENT_NO"])
#rename the columns (inplace)
wrk_accidentevent = wrk_accidentevent.rename(columns={'Event Type Desc': 'CATEventTypeDesc_accevent',
                                                      'Object Type Desc': 'CATObjectTypeDesc_accevent' })

In [21]:
#Left join
wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_3 = pd.merge(wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_2, wrk_road_surface_condition, how="left", on=["ACCIDENT_NO"])
wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_4 = pd.merge(wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_3, wrk_subDCA, how="left", on=["ACCIDENT_NO"])
wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_5 = pd.merge(wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_4, wrk_amosphericcond, how="left", on=["ACCIDENT_NO"])
wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_6 = pd.merge(wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_5, wrk_accidentlocation, how="left", on=["ACCIDENT_NO"])
wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_7 = pd.merge(wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_6, wrk_accidentevent, how="left", on=["ACCIDENT_NO"])

#rename the columns (inplace)
wrk_final_accident_pedestrians_melbourne_2008_2020 = wrk_inner_merge_Accident_Pedestrians_Melbourne_2008_2020_7.rename(columns={'ACCIDENT_NO': 'KEYAccidentNumber'})

wrk_final_accident_pedestrians_melbourne_2008_2020.head()

,KEYAccidentNumber,DATAccidentDate_accident,TIMAccidentTime_accident,CATAccidentTypeDesc_accident,CATDayOfWeek_accident,CATDCADesc_accident,CATMelwaysPage_accident,CATMelwaysGridRef_X_accident,CATMelwaysGridRef_Y_accident,CATLightConditionDesc_accident,...,CATPostcode_node,CATSurfaceConditionDesc_surface,CATSubDCACodeDesc_subdca,CATAtmosphericConditionDesc_atmosphere,CATRoadName_acclocation,CATRoadNameInt_acclocation,CATRoadType_acclocation,CATRoadTypeInt_acclocation,CATEventTypeDesc_accevent,CATObjectTypeDesc_accevent
0,T20080000929,2008-01-09,08:10:00,Struck Pedestrian,Wednesday,FAR SIDE. PED HIT BY VEHICLE FROM THE LEFT ...,1B,P,10,Day,...,3000,Dry,Not Required,Clear,FLINDERS,CHAPTER HOUSE,STREET,LANE,Collision,Not Applicable
1,T20080001163,2008-01-11,15:35:00,Collision with vehicle,Friday,REAR END(VEHICLES IN SAME LANE),2E,B,1,Day,...,3003,Dry,Mid-block,Clear,FOOTSCRAY,CITYLINK,ROAD,NaN,Collision,Not Applicable
2,T20080001766,2008-01-16,14:24:00,Struck Pedestrian,Wednesday,PED NEAR SIDE. PED HIT BY VEHICLE FROM THE RIG...,1B,V,3,Day,...,3000,Dry,Vehicle leaving intersection,Clear,LONSDALE,SPRING,STREET,STREET,Collision,Not Applicable
3,T20080002335,2008-01-19,20:00:00,Struck Pedestrian,Saturday,ANY MANOEUVRE INVOLVING PED NOT INCLUDED IN DC...,2F,H,8,Dusk/Dawn,...,3004,Dry,Not Required,Clear,LINLITHGOW,LINLITHGOW ALT,AVENUE,AVENUE,Collision,Not Applicable
4,T20080002388,2008-01-21,14:45:00,Struck Pedestrian,Monday,PED NEAR SIDE. PED HIT BY VEHICLE FROM THE RIG...,2A,F,9,Day,...,3051,Dry,Vehicle leaving intersection,Clear,DRYBURGH,QUEENSBERRY,STREET,STREET,Collision,Not Applicable


In [22]:
wrk_final_accident_pedestrians_melbourne_2008_2020.shape

(2028, 34)

In [23]:
list(wrk_final_accident_pedestrians_melbourne_2008_2020)

['KEYAccidentNumber',
 'DATAccidentDate_accident',
 'TIMAccidentTime_accident',
 'CATAccidentTypeDesc_accident',
 'CATDayOfWeek_accident',
 'CATDCADesc_accident',
 'CATMelwaysPage_accident',
 'CATMelwaysGridRef_X_accident',
 'CATMelwaysGridRef_Y_accident',
 'CATLightConditionDesc_accident',
 'NUMVehiclesInvolved_accident',
 'NUMPersonsInvolved_accident',
 'NUMPersonsInjured_accident',
 'KEYPersonID_person',
 'CATRoadUserTypeDesc_person',
 'CATTakenHospital_person',
 'CATInjuryLevelDesc_person',
 'CATAgeGroup_person',
 'CATPostcode_person',
 'CATGender_person',
 'CATLGAName_node',
 'CATDEGUrbanName_node',
 'NUMLatitude_node',
 'NUMLongitude_node',
 'CATPostcode_node',
 'CATSurfaceConditionDesc_surface',
 'CATSubDCACodeDesc_subdca',
 'CATAtmosphericConditionDesc_atmosphere',
 'CATRoadName_acclocation',
 'CATRoadNameInt_acclocation',
 'CATRoadType_acclocation',
 'CATRoadTypeInt_acclocation',
 'CATEventTypeDesc_accevent',
 'CATObjectTypeDesc_accevent']

In [24]:
#Save final dataset as CSV to google drive
if 'google.colab' in str(get_ipython()):
    from google.colab import drive

    # Mount your Drive to the Colab VM.
    drive.mount('/gdrive')

    # Write the DataFrame to CSV file.
    with open('/gdrive/My Drive/Uni Assignments/SIT782 Team Project B/data/Accidents_Pedestrians_Melbourne_2008to2020.csv', 'w') as f:
        wrk_final_accident_pedestrians_melbourne_2008_2020.to_csv(f, index=False)
        #write the csv without index row values.
else:
    wrk_final_accident_pedestrians_melbourne_2008_2020.to_csv('search-results.csv', index=False)

Mounted at /gdrive
